### Household Characteristics

In [28]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [29]:
import pandas as pd
import numpy as np
from os.path import dirname, join
import os

#bokeh modules
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure,gmap
from bokeh.core.properties import value
from bokeh.transform import factor_cmap, dodge
from bokeh.layouts import layout, column, row, WidgetBox
from bokeh.models import Panel, Spacer, HoverTool, ColumnDataSource, FactorRange,NumeralTickFormatter
from bokeh.models.widgets import Div, Tabs, Paragraph, Dropdown, Button, PreText, Toggle, TableColumn, DataTable

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()

Loading BokehJS ...

In [30]:
full_width = 965
census_color = "#EFF1EF"
survey_color = '#9EA499'
cmap_color = '#495667'



survey_dir = join(os.getcwd(),'data','survey')
model_dir = join(os.getcwd(),'data','model')
ctpp_dir =  join(os.getcwd(),'data','ctpp')

ao_counts = pd.read_csv(join(os.getcwd(),'data','aoCounts.csv'),index_col=[0])
person_trips = pd.read_csv(join(os.getcwd(),'data','model','person_trips.csv'))
survey_trips = pd.read_csv(join(os.getcwd(),'data','survey','TravelSurvey_weekday_rsg.csv'))

In [31]:
def test_tab():

    h_1 = Div(text = """<h1><center>Intro Text</center></h1>""")
    
    l_1 = layout(children=[h_1])

    tab_1 = Panel(child=l_1, title = '# Introduction')
    
    return tab_1

In [32]:
def AutoOwnership(ao_counts,survey_dir,model_dir,ctpp_dir):

    full_width = 965
    census_color = "#EFF1EF"
    survey_color = '#9EA499'
    cmap_color = '#495667' 

    survey_income = pd.read_csv(os.path.join(survey_dir,'income_survey.csv'))
    survey_size = pd.read_csv(os.path.join(survey_dir,'size_survey.csv'))
    survey_workers = pd.read_csv(os.path.join(survey_dir,'workers_survey.csv'))
    
    ctpp_income = pd.read_csv(os.path.join(ctpp_dir,'income_ctpp.csv'))
    ctpp_size = pd.read_csv(os.path.join(ctpp_dir,'size_ctpp.csv'))
    ctpp_workers = pd.read_csv(os.path.join(ctpp_dir,'workers_ctpp.csv'))
    
    model_income = pd.read_csv(os.path.join(model_dir,'income_model.csv'))
    model_size = pd.read_csv(os.path.join(model_dir,'size_model.csv'))
    model_workers = pd.read_csv(os.path.join(model_dir,'workers_model.csv'))


    #graphic functions
    TOOLS = "reset,hover,save"
    
    #make src data
    def make_src(df):
        
        groupby = df.index.values.tolist()
        census = df['Census'].values.tolist()
        survey = df['Survey'].values.tolist()
        model = df['Model'].values.tolist()
        
        data = {'Group': groupby,
               'Census': census,
               'Survey': survey,
               'Model': model}
        
        return ColumnDataSource(data=data), groupby, data
    
    #make groupbed bar chart
    def makeGroupBar(src, groups, data):
        
        p = figure(x_range=FactorRange(*groups),title="Figure 1 - Auto Ownership Distribution",
                   plot_width = full_width, plot_height = full_width/4,
                   tools="hover",toolbar_location = None)
        
        p.vbar(x=dodge('Group',-0.25,range=p.x_range),top='Census', width=0.25, source=src,
               color=census_color, legend=value("Census"))

        p.vbar(x=dodge('Group',0,range=p.x_range),top='Survey', width=0.25, source=src,
               color=survey_color, legend=value("Survey"))
        
        p.vbar(x=dodge('Group',0.25,range=p.x_range),top='Model', width=0.25, source=src,
               color=cmap_color, legend=value("Model"))
        
        p.select_one(HoverTool).tooltips = [
             ('Census - Household Total',"@Census{0,}"),
             ('Survey - Household Total',"@Survey{0,}"),
             ('Model - Household Total',"@Model{0,}")
        ]
        
        # Styling
        p = style(p)
        
        p.legend.click_policy="hide"

        return p 
    
    def style(p):
        
        p.xgrid.visible = False
        p.ygrid.visible = False
        
        p.yaxis.formatter = NumeralTickFormatter(format="0,")
            
        return p
    
    
    #css_classes= ["well"], 
    h_2 = Div(text = """<h2># Auto Ownership</h2>
                <p>Contrary to popular belief, Lorem Ipsum is not simply random text. 
                It has roots in a piece of classical Latin literature from 45 BC, 
                making it over 2000 years old. Richard McClintock, a Latin professor 
                at Hampden-Sydney College in Virginia, looked up one of the more obscure 
                Latin words, consectetur, from a Lorem Ipsum passage, and going through 
                the cites of the word in classical literature, discovered the undoubtable 
                source. Lorem Ipsum comes from sections 1.10.32 and 1.10.33 of "de Finibus 
                Bonorum et Malorum" (The Extremes of Good and Evil) by Cicero, written in 
                45 BC. This book is a treatise on the theory of ethics, very popular during 
                the Renaissance. The first line of Lorem Ipsum, "Lorem ipsum dolor sit 
                amet..", comes from a line in section 1.10.32.</p>
                <p>The standard chunk of Lorem Ipsum used since the 1500s is reproduced below 
                for those interested. Sections 1.10.32 and 1.10.33 from "de Finibus Bonorum et 
                Malorum" by Cicero are also reproduced in their exact original form, accompanied 
                by English versions from the 1914 translation by H. Rackham.</p>""", 
                width = full_width, sizing_mode='stretch_both',
                style={"width":'100%',"text-align":'left',"margin":'0 auto'})
    
    #Auto Ownership Graph
    ao_src = make_src(ao_counts)
    ao_graph = makeGroupBar(ao_src[0], ao_src[1], ao_src[2])
    
    h_2_1 = row(ao_graph,width = 965)
    
    #---------------------------------------------------------------------------------------------------------------
    #Auto Ownership by Household Characteristic Tables
    h_2_2 = Div(text = """<h2># | Auto Ownership By Household Characteristics</h2>
                <p>ontrary to popular belief, Lorem Ipsum is not simply random text. It 
                has roots in a piece of classical Latin literature from 45 BC, making it 
                over 2000 years old. Richard McClintock, a Latin professor at Hampden-Sydney 
                College in Virginia, looked up one of the more obscure Latin words, 
                consectetur, from a Lorem Ipsum passage, and going through the cites 
                of the word in classical literature, discovered the undoubtable source. 
                Lorem Ipsum comes from sections 1.10.32 and 1.10.33 of "de Finibus Bonorum 
                et Malorum" (The Extremes of Good and Evil) by Cicero, written in 45 BC.</p>""", 
                width = full_width, sizing_mode='stretch_both',
                style={"width":'100%',"text-align":'left',"margin":'0 auto'})
    
    
    #Tables
    #ADD FORMATS
    tbl_income = Div(text="<h4><center>Auto Ownership By Household Income</center></h4>",width=full_width)
    
    s1_html = Div(text="<h5>Table 1 - Survey Income</h5>"+survey_income.to_html(index=False, classes=["table-bordered", "table-striped", "table-hover"], 
                                    justify='center', float_format='{:20,.1f}%'.format))
    
    c1_html = Div(text="<h5>Table 2 - CTPP Income</h5>"+ctpp_income.to_html(index=False, classes=["table-bordered", "table-striped", "table-hover"], 
                                    justify='center', float_format='{:20,.1f}%'.format)+"<br>")
    
    m1_html = Div(text="<h5>Table 3 - Model Income</h5>"+model_income.to_html(index=False, classes=["table-bordered", "table-striped", "table-hover"], 
                                    justify='center', float_format='{:20,.1f}%'.format)+"<br>")
    
    
    tbl_size = Div(text="<h4><center>Auto Ownership By Household Size</center></h4>",width=full_width)
    
    s2_html = Div(text="<h5>Table 4 - Survey Household Size</h5>"+survey_size.to_html(index=False, classes=["table-bordered", "table-striped", "table-hover"], 
                                    justify='center', float_format='{:20,.1f}%'.format)+"<br>")
    
    c2_html = Div(text="<h5>Table 5 - CTPP Household Size</h5>"+ctpp_size.to_html(index=False, classes=["table-bordered", "table-striped", "table-hover"], 
                                    justify='center', float_format='{:20,.1f}%'.format)+"<br>")
    
    m2_html = Div(text="<h5>Table 6 - Model Household Size</h5>"+model_size.to_html(index=False, classes=["table-bordered", "table-striped", "table-hover"], 
                                    justify='center', float_format='{:20,.1f}%'.format)+"<br>")
    
    tbl_workers = Div(text="<h4><center>Auto Ownership By Workers In Household</center></h4>",width=full_width)
    
    s3_html = Div(text="<h5>Table 7 - Survey Workers</h5>"+survey_workers.to_html(index=False, classes=["table-bordered", "table-striped", "table-hover"], 
                                    justify='center', float_format='{:20,.1f}%'.format)+"<br>")
    
    c3_html = Div(text="<h5>Table 8 - CTPP Workers</h5>"+ctpp_workers.to_html(index=False, classes=["table-bordered", "table-striped", "table-hover"], 
                                    justify='center', float_format='{:20,.1f}%'.format)+"<br>")
    
    m3_html = Div(text="<h5>Table 9 - Model Workers</h5>"+model_workers.to_html(index=False, classes=["table-bordered", "table-striped", "table-hover"], 
                                    justify='center', float_format='{:20,.1f}%'.format)+"<br>")
    
    #tbl_survey = Div(text=s1_html+"<br>"+s2_html+"<br>"+s3_html)
    
    #tbl_census = Div(text=s1_html+"<br>"+s2_html+"<br>"+s3_html)
    
    ao = [h_2, h_2_1,Div(text="<hr>",width=full_width),h_2_2,row(tbl_income),
                           row(s1_html,Spacer(width=50),c1_html,Spacer(width=50),m1_html),
                           row(tbl_size),
                           row(s2_html,Spacer(width=50),c2_html,Spacer(width=50),m2_html),
                           row(tbl_workers),
                           row(s3_html,Spacer(width=50),c3_html,Spacer(width=50),m3_html),
        Div(text="<hr>",width=full_width)]
    
    return ao

In [33]:
def Flow():
     
    h_3 = Div(text = """<h1># Flow Patterns</h1>
                <p>Contrary to popular belief, Lorem Ipsum is not simply random text. 
                It has roots in a piece of classical Latin literature from 45 BC, 
                making it over 2000 years old. Richard McClintock, a Latin professor 
                at Hampden-Sydney College in Virginia, looked up one of the more obscure 
                Latin words, consectetur, from a Lorem Ipsum passage, and going through 
                the cites of the word in classical literature, discovered the undoubtable 
                source. Lorem Ipsum comes from sections 1.10.32 and 1.10.33 of "de Finibus 
                Bonorum et Malorum" (The Extremes of Good and Evil) by Cicero, written in 
                45 BC. This book is a treatise on the theory of ethics, very popular during 
                the Renaissance. The first line of Lorem Ipsum, "Lorem ipsum dolor sit 
                amet..", comes from a line in section 1.10.32.</p>
                <p>The standard chunk of Lorem Ipsum used since the 1500s is reproduced below 
                for those interested. Sections 1.10.32 and 1.10.33 from "de Finibus Bonorum et 
                Malorum" by Cicero are also reproduced in their exact original form, accompanied 
                by English versions from the 1914 translation by H. Rackham.</p><hr>""", 
                width = full_width, sizing_mode='stretch_both',
                style={"width":'100%',"text-align":'left',"margin":'0 auto'})
    
    l_1 = [h_3]

    return l_1

In [34]:
def TripDist():
     
        
    #Trip Distance
    
        
        
    h_1 = Div(text = """<h1># Trip Distance</h1>
                <p>Contrary to popular belief, Lorem Ipsum is not simply random text. 
                It has roots in a piece of classical Latin literature from 45 BC, 
                making it over 2000 years old. Richard McClintock, a Latin professor 
                at Hampden-Sydney College in Virginia, looked up one of the more obscure 
                Latin words, consectetur, from a Lorem Ipsum passage, and going through 
                the cites of the word in classical literature, discovered the undoubtable 
                source. Lorem Ipsum comes from sections 1.10.32 and 1.10.33 of "de Finibus 
                Bonorum et Malorum" (The Extremes of Good and Evil) by Cicero, written in 
                45 BC. This book is a treatise on the theory of ethics, very popular during 
                the Renaissance. The first line of Lorem Ipsum, "Lorem ipsum dolor sit 
                amet..", comes from a line in section 1.10.32.</p><hr>""", 
                width = full_width, sizing_mode='stretch_both',
                style={"width":'100%',"text-align":'left',"margin":'0 auto'})
    
    
    
    l_1 = [h_1]

    return l_1

In [35]:
def ModeChoice(trips_csv, survey):

    full_width = 965
    census_color = "#EFF1EF"
    survey_color = '#9EA499'
    cmap_color = '#495667'
    
    def filter_trips(trips, mode_select, purp_select, pertype_select):
        
        if mode_select == 'All' or mode_select is None:
            m = trips.trip_mode.drop_duplicates().values.tolist()
        else:
            m = [mode_select]
            
        if purp_select == 'All' or purp_select is None:
            p = trips.trip_purpose.drop_duplicates().values.tolist()
        else:
            p= [purp_select]
            
        if pertype_select == 'All' or pertype_select is None:
            t = trips.type.drop_duplicates().values.tolist()
        else:
            t= [pertype_select]
        
        
        select_trips =  trips.loc[(trips['trip_purpose'].isin(p)) & (trips['type'].isin(t))] 
        survey_trips = survey.loc[(survey['trip_purpose'].isin(p)) & (trips['type'].isin(t))]
        
        mtrips = select_trips.groupby('trip_mode').agg({'Model':sum})
        strips = survey_trips.groupby('trip_mode').agg({'Weight2':sum})
        strips.columns = ['Survey']
        
        trips = mtrips.merge(strips, how='left', left_index=True, right_index=True)
        
        
        
        numtrips = "{:,}".format(select_trips.Model.sum())
        
        return trips

    
    #graphic functions
    TOOLS = "reset,hover,save"
    
    #make src data
    def make_src(df):
        
        df.sort_values(by='Survey',ascending=False, inplace=True)
        
        groupby = df.index.values.tolist()
        survey = df['Survey'].values.tolist()
        model = df['Model'].values.tolist()
        
        data = {'Group': groupby,
               'Observed': survey,
               'Model': model}
        
        return ColumnDataSource(data=data)
    
    #make groupbed bar chart
    def makeGroupBar(src):
        
        groups = trips_csv.trip_mode.drop_duplicates().values.tolist()
        
        p = figure(x_range=FactorRange(*groups),
                   plot_width = full_width, plot_height = full_width/4,
                   tools="hover",toolbar_location = None)

        p.vbar(x=dodge('Group',-0.25,range=p.x_range),top='Observed', width=0.25, source=src,
               color=survey_color, legend=value("Observed"))
        
        p.vbar(x=dodge('Group',0,range=p.x_range),top='Model', width=0.25, source=src,
               color=cmap_color, legend=value("Model"))
        
        p.select_one(HoverTool).tooltips = [
             ('Observed - Trip',"@Observed{0,}"),
             ('Model - Trip',"@Model{0,}")
        ]
        
        # Styling
        p = style(p)
        
        p.legend.click_policy="hide"

        return p 
    
    def style(p):
        
        p.xgrid.visible = False
        p.ygrid.visible = False
        
        p.yaxis.formatter = NumeralTickFormatter(format="0,")
            
        return p
    
    def update(attr, old, new):

        p_select, type_select = purpose_selection.value,  type_selection.value

        purposeTrips = filter_trips(trips_csv,'All',p_select,'All')  
        perTypeTrips = filter_trips(trips_csv,'All','All', type_select)

        #numtrips.text = ftrips[0] + " | <text style='color:Gray'>100</div> Trips"

        new_src_purp = make_src(purposeTrips)
        src_purp.data.update(new_src_purp.data)
        
        new_src_type = make_src(perTypeTrips)
        src_type.data.update(new_src_type.data)
        
        purp_title.text = "<h5>Figure # - %s Trips by Mode (Select trip purpose from dropdown)</h5>" % (p_select)
        type_title.text = "<h5>Figure # - %s Trips by Mode (Select person type from dropdown)</h5>" % (type_select)
    
    #Dropdown#---------------------------------------------------------------------------------------------------
    purpose_menu = [('All','All'),
             ('Discretionary','Discretionary'),
             ('Eating Out','Eating Out'),
             ('Escort','Escort'),
             ('Maintenance','Maintenance'),
             ('School','School'),
             ('Shop','Shopping'),
             ('University','University'),
             ('Visiting','Visiting'),
             ('Work','Work'),
             ('Work-Based','Work-Based')]
    
    type_menu = [('All','All'),
             ('Child Too Young For School','Child too young for school'),
             ('Full-Time Worker','Full-time worker'), 
             ('Non-Worker','Non-worker'),  
             ('Part-Time Worker','Part-time worker'),    
             ('Retired','Retired'),
             ('Student Of Driving Age','Student of driving age'),
             ('Student Of Non-Driving Age','Student of non-driving age')]
    
    #Widgets
    purpose_selection = Dropdown(label="Trip Purpose", button_type="default", 
                                 menu=purpose_menu, width=250)
    
    type_selection = Dropdown(label="Person Type", button_type="default", 
                                 menu=type_menu, width=250)
    
    #onchange
    purpose_selection.on_change('value', update)
    type_selection.on_change('value', update)
    
    #create initial bar chart
    purposeTrips = filter_trips(trips_csv,'All','All','All')
    perTypeTrips = filter_trips(trips_csv,'All','All','All')
    
    #if purpose_selection.value == None:
    #    purpose_selection.value = 'All'
        
    purp_title = Div(text= "<h5>Figure # - %s Trips by Mode (Select trip purpose from dropdown)</h5>" % ('All'),
                   width = full_width)
    
    type_title = Div(text= "<h5>Figure # - %s Trips by Mode (Select person type from dropdown)</h5>" % ('All'),
                   width = full_width)
    
    src_purp = make_src(purposeTrips)
    purpBarChart = makeGroupBar(src_purp)
    
    src_type = make_src(perTypeTrips)
    typeBarChart = makeGroupBar(src_type)
      
    #---------------------------------------------------------------------------------------------------------------
    
    h_4 = Div(text = """<h1># Trip Mode Choice</h1>
                <p>Contrary to popular belief, Lorem Ipsum is not simply random text. 
                It has roots in a piece of classical Latin literature from 45 BC, 
                making it over 2000 years old. Richard McClintock, a Latin professor 
                at Hampden-Sydney College in Virginia, looked up one of the more obscure 
                Latin words, consectetur, from a Lorem Ipsum passage, and going through 
                the cites of the word in classical literature, discovered the undoubtable 
                source. Lorem Ipsum comes from sections 1.10.32 and 1.10.33 of "de Finibus 
                Bonorum et Malorum" (The Extremes of Good and Evil) by Cicero, written in 
                45 BC. This book is a treatise on the theory of ethics, very popular during 
                the Renaissance. The first line of Lorem Ipsum, "Lorem ipsum dolor sit 
                amet..", comes from a line in section 1.10.32.</p>
                <p>The standard chunk of Lorem Ipsum used since the 1500s is reproduced below 
                for those interested. Sections 1.10.32 and 1.10.33 from "de Finibus Bonorum et 
                Malorum" by Cicero are also reproduced in their exact original form, accompanied 
                by English versions from the 1914 translation by H. Rackham.</p>""", 
                width = full_width, sizing_mode='stretch_both',
                style={"width":'100%',"text-align":'left',"margin":'0 auto'})
    
    source = Div(text="""<small><center>*Observed Trips: <a href="http://www.cmap.illinois.gov/data/transportation/travel-survey">
                2007-08 Travel Tracker Survey</a></center></small>""",width = full_width)
       
    
    l_1 = [h_4,Spacer(height=25),row(purpose_selection,width=full_width),
           purp_title,purpBarChart,Spacer(height=25),
           row(type_selection,width=full_width),
           type_title,typeBarChart,source,Div(text="<hr>",width=full_width)]

    return l_1

In [36]:
def TripPurpose(trips,survey):

    full_width = 965
    census_color = "#EFF1EF"
    survey_color = '#9EA499'
    cmap_color = '#495667'

    def tripsXpurpose(trips,survey):
        mtrips = trips.groupby('trip_purpose').agg({'Model':sum})
        strips = survey.groupby('trip_purpose').agg({'Weight2':sum})
        strips.columns = ['Survey']

        trips = strips.merge(mtrips, how='left', left_index=True, right_index=True)
        
        trips.loc[:,'Model Share'] = ((trips['Model']/trips['Model'].sum()) *100)
        trips.loc[:,'Survey Share'] = ((trips['Survey']/trips['Survey'].sum()) *100)
        trips.loc['Total'] = trips.sum()
        
        shares = trips[['Model Share','Survey Share']]
        shares.columns = ['Model','Survey']
        shares.loc[:,'Difference'] = (shares['Model'] - shares['Survey'])
        shares.loc[:,'% Difference'] = ((shares['Difference']/shares['Survey']) * 100)
        
        trips.loc[:,'Difference'] = (trips['Model'] - trips['Survey']).round(1)
        trips.loc[:,'% Difference'] = ((trips['Difference']/trips['Survey']) * 100)


        trip_purpose = trips[['Survey','Model','Difference','% Difference']].reset_index()
        trip_purpose.columns = ['Purpose','Observed','Model','Difference','% Difference']

        purpose_shares = shares[['Survey','Model','Difference','% Difference']].reset_index()
        purpose_shares.columns = ['Purpose','Observed','Model','Difference','% Difference']

        return trip_purpose, purpose_shares    

    
    def filter_trips(trips, pertype_select):
            
        if pertype_select == 'All' or pertype_select is None:
            t = trips['type'].drop_duplicates().values.tolist()
        else:
            t= [pertype_select]
        
        
        select_trips =  trips.loc[(trips['type'].isin(t))] 
        survey_trips = survey.loc[(survey['type'].isin(t))]
        
        mtrips = select_trips.groupby('trip_purpose').agg({'Model':sum})
        strips = survey_trips.groupby('trip_purpose').agg({'Weight2':sum})
        strips.columns = ['Survey']
        
        trips_ = mtrips.merge(strips, how='left', left_index=True, right_index=True)
        
        return trips_

    #graphic functions
    TOOLS = "reset,hover,save"
    
    #make src data
    def make_src(df):
        
        df.sort_values(by='Survey',ascending=False, inplace=True)
        
        groupby = df.index.values.tolist()
        survey = df['Survey'].values.tolist()
        model = df['Model'].values.tolist()
        
        data = {'Group': groupby,
               'Observed': survey,
               'Model': model}
        
        return ColumnDataSource(data=data)
    
    #make groupbed bar chart
    def makeGroupBar(src):
        
        groups = trips.trip_mode.drop_duplicates().values.tolist()
        
        p = figure(x_range=FactorRange(*groups),
                   plot_width = full_width, plot_height = full_width/4,
                   tools="hover",toolbar_location = None)

        p.vbar(x=dodge('Group',-0.25,range=p.x_range),top='Observed', width=0.25, source=src,
               color=survey_color, legend=value("Observed"))
        
        p.vbar(x=dodge('Group',0,range=p.x_range),top='Model', width=0.25, source=src,
               color=cmap_color, legend=value("Model"))
        
        p.select_one(HoverTool).tooltips = [
             ('Observed - Trips',"@Observed{0,}"),
             ('Model - Trips',"@Model{0,}")
        ]
        
        # Styling
        p = style(p)
        
        p.legend.click_policy="hide"

        return p 
    
    def style(p):
        
        p.xgrid.visible = False
        p.ygrid.visible = False
        
        p.yaxis.formatter = NumeralTickFormatter(format="0,")
            
        return p
    
    def update(attr, old, new):

        type_select = type_selection.value

        perTypeTrips = filter_trips(trips, type_select)
        
        new_src_type = make_src(perTypeTrips)
        src_type.data.update(new_src_type.data)

        type_title.text = "<h5>Figure # - %s Trips by Mode (Select person type from dropdown)</h5>" % (type_select)
    
    #Dropdown#---------------------------------------------------------------------------------------------------
    type_menu = [('All','All'),
             ('Child Too Young For School','Child too young for school'),
             ('Full-Time Worker','Full-time worker'), 
             ('Non-Worker','Non-worker'),  
             ('Part-Time Worker','Part-time worker'),    
             ('Retired','Retired'),
             ('Student Of Driving Age','Student of driving age'),
             ('Student Of Non-Driving Age','Student of non-driving age')]
    
    #Widgets
    type_selection = Dropdown(label="Person Type", button_type="default", 
                                 menu=type_menu, width=250)
    
    #onchange
    type_selection.on_change('value', update)
    
    #create initial bar chart
    perTypeTrips = filter_trips(trips,'All')
    
    #if purpose_selection.value == None:
    #    purpose_selection.value = 'All'
        
    type_title = Div(text= "<h5>Figure # - %s Trips by Purpose (Select person type from dropdown)</h5>" % ('All'),
                   width = full_width)
    
    src_type = make_src(perTypeTrips)
    typeBarChart = makeGroupBar(src_type) 
        
    #Trip Purpose
    ptrips = tripsXpurpose(trips,survey)
    trip_purpose = ptrips[0]
    share = ptrips[1]

    purp_html = Div(text="<h5>Table # - Trips by Purpose</h5>"+trip_purpose.to_html(index=False, classes=["table-bordered", "table-striped", "table-hover"], 
                                    justify='center', float_format='{:20,.1f}'.format),width = full_width/3)
    
    purpshare_html = Div(text="<h5>Table # - Trip Share by Purpose</h5>"+share.to_html(index=False, classes=["table-bordered", "table-striped", "table-hover"], 
                                    justify='center', float_format='{:20,.1f}%'.format),width = full_width/3)
    
    source = Div(text="""<small><center>*Observed Trips: <a href="http://www.cmap.illinois.gov/data/transportation/travel-survey">
                    2007-08 Travel Tracker Survey</a></center></small>""",width = full_width)
        
        
    h_1 = Div(text = """<h1># Calibrating Travel Purpose</h1>
                <p>Contrary to popular belief, Lorem Ipsum is not simply random text. 
                It has roots in a piece of classical Latin literature from 45 BC, 
                making it over 2000 years old. Richard McClintock, a Latin professor 
                at Hampden-Sydney College in Virginia, looked up one of the more obscure 
                Latin words, consectetur, from a Lorem Ipsum passage, and going through 
                the cites of the word in classical literature, discovered the undoubtable 
                source. Lorem Ipsum comes from sections 1.10.32 and 1.10.33 of "de Finibus 
                Bonorum et Malorum" (The Extremes of Good and Evil) by Cicero, written in 
                45 BC. This book is a treatise on the theory of ethics, very popular during 
                the Renaissance. The first line of Lorem Ipsum, "Lorem ipsum dolor sit 
                amet..", comes from a line in section 1.10.32.</p>""", 
                width = full_width, sizing_mode='stretch_both',
                style={"width":'100%',"text-align":'left',"margin":'0 auto'})
    
    
    
    l_1 = [h_1,row(Spacer(width=50),purp_html,Spacer(width=full_width/5),purpshare_html,width=full_width),
          type_selection,source,Div(text="<hr>",width=full_width)]

    return l_1

In [37]:
def Calibration_tab(ao_counts,survey_dir,model_dir,ctpp_dir,person_trips,survey_trips):
    
    ao = AutoOwnership(ao_counts,survey_dir,model_dir,ctpp_dir)
    flow = Flow()
    modechoice = ModeChoice(person_trips,survey_trips)
    tpurp = TripPurpose(person_trips,survey_trips)
    
    l_2 = layout(children=ao+flow+modechoice+tpurp)

    return Panel(child=l_2, title = '# Model Calibration')

In [38]:
def Validation_tab():
    
    h_1 = Div(text = """<h1><center>Intro Text</center></h1>""")
    
    l_1 = layout(children=[h_1])

    return Panel(child=l_1, title = '# Validation')

In [39]:
def abm_report(doc):
    
    tab_1 = test_tab()
    tab_2 = Calibration_tab(ao_counts,survey_dir,model_dir,ctpp_dir,person_trips,survey_trips)
    tab_3 = Validation_tab()
   
    tabs = Tabs(tabs=[tab_2,tab_3])
    
    doc.add_root(tabs)
        
handler = FunctionHandler(abm_report)
app = Application(handler)
show(app)